<a href="https://colab.research.google.com/github/raf-madrigal/nlp-sandbox/blob/main/data_cleaning_using_textanalyzer_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Text EDA
- Distribution of text lenghts
- Word count
- Distribution of top unigrams before removing stopwords (1, 2, 3(
- after removing stopwords
- Distribution of POS Tags
- Review lenght per cateogry
- sentiment distribution
-

In [ ]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(df['Review Text'], 20)
for word, freq in common_words:
    print(word, freq)
df1 = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
df1.groupby('ReviewText').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black', title='Top 20 words in review before removing stop words')


In [ ]:
import re
import string
import unicodedata
from collections import Counter, defaultdict

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

stop_en = nltk.corpus.stopwords.words('english')

CONTRACTIONS = {
    "ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
    "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
    "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
    "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
    "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
    "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
    "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
    "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
    "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
    "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
    "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
    "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
    "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
    "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
    "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
    "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
    "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
    "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
    "you're": "you are", "you've": "you have"}


class TextAnalyzer:
    """Class contains methods that does text analysis"""

    def __init__(self, stopwords=[], clean=False, remove_digits=True,
                 min_wordlen=2):
        """Constructor method

        Parameters
        ----------
        stopwords: set/list
            list of stopwords
        clean : bool
            Flag whether to run cleaning methods
        remove_digits : bool
            Whether to remove digits

        """
        self.clean = clean
        self.remove_digits = remove_digits
        self.min_wordlen = min_wordlen
        self.stopwords = set(nltk.corpus.stopwords.words('english') +
                             stopwords)

    def fit(self, text_file):
        self.text = text_file
        if self.clean:
            self._expand_contractions()
            self._remove_ownership()
            self._remove_comments_in_parenthesis()
            self._remove_accented()
            self._remove_punctuation()
            self._remove_special(remove_digits=self.remove_digits)
            self._lowercase_all()
            self._remove_stopwords()
            self._remove_short_words(min_wordlen=self.min_wordlen)


        else:
            self.text = '\n'.join(list(filter(lambda x: x.strip(), self.text)))

        return self
    def _remove_short_words(self, min_wordlen):
        s = self.text
        self.text = (' '.join([word for word in s.split()
                                   if len(word)>=min_wordlen]).strip())
    def _remove_comments_in_parenthesis(self):
        s = self.text
        self.text = re.sub(r'\([^)]*\)', '', s)

    def _remove_ownership(self):
        s = self.text
        self.text = re.sub(r"'s\b", "", s)

    def _expand_contractions(self, contractions_dict=CONTRACTIONS):
        s = self.text
        contractions_re = re.compile(f'{"|".join(contractions_dict.keys())}')

        def replace(match):
            return contractions_dict[match.group(0)]

        self.text = (' '.join([contractions_re.sub(replace, x, re.I)
                                if x in contractions_dict.keys()
                                else x
                                for x in s.split(' ')]))

    def _remove_accented(self):
        text = self.text
        special_char_pattern = re.compile(r'([{.(-)!}])')
        text = special_char_pattern.sub(" \\1 ", text)
        text = (unicodedata.normalize('NFKD', text)
                           .encode('ascii', 'ignore')
                           .decode('utf-8', 'ignore'))
        self.text = text

    def _remove_special(self, remove_digits=True):
        s = self.text
        if remove_digits:
            self.text = re.sub(r'[^a-zA-Z]', ' ', s)
        else:
            self.text = re.sub(r'[^a-zA-Z0-9]', ' ', s)


    def _join_sentences(self):
        """Internal method for joining sentences

        Checks and groups lines/sentences per paragraph. Paragraphs
        are separated by '\n'.
        """

        par_list = self.text.split('\n')

        sentences = []
        paragraph = []
        for par in par_list:
            sentences = ' '.join(par.split()).strip()
            paragraph.append(sentences)

        self.text = '\n'.join(par_list).strip()

    def _remove_punctuation(self, punct=string.punctuation):
        """Internal method for removing punctuations

        Punctuations marks are defined by globally using the
        list PUNCTUATION_MARKS. This method checks for each
        punctuation and replaces them as an empty string
        """
        text = self.text
        # punct = [i for i in punct if i is not '.']
        self.text = text.translate(str.maketrans(punct,
                                     ''*len(punct)))

    def _remove_stopwords(self):
        """Internal method for removing stopwords"""

        par_list = []
        for par in self.text.split('\n'):
            sent = ' '.join([word for word in par.split()
                                  if word not in self.stopwords])
            par_list.append(sent)

        self.text = '\n'.join(par_list).strip()

    def _lowercase_all(self):
        """Internal method for converting text to lower case"""
        self.text = self.text.lower()

    def stem(self):
        text = self.text
        ps = nltk.porter.PorterStemmer()
        text = ' '.join([ps.stem(w) for w in nltk.word_tokenize(text)])
        self.text = text
        return self

    def lemmatize(self):
        pars = self.text

        def get_wordnet_pos(word):
            """Map POS tag to first character lemmatize() accepts"""
            tag = nltk.pos_tag([word])[0][1][0].upper()
            tag_dict = {"J": wordnet.ADJ,
                        "N": wordnet.NOUN,
                        "V": wordnet.VERB,
                        "R": wordnet.ADV}

            return tag_dict.get(tag, wordnet.NOUN)

        lemmatizer = WordNetLemmatizer()
        par_compile = []
        for par in pars.split('\n'):
            par_compile.append(' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(par)]))
        self.text = '\n'.join(par_compile)
        return self

    def ngrams(self, n=1):
        """Returns the n-gram of the text.

        An 𝑛-gram is a sequence of 𝑛 successive words in a paragraph of text.

        Parameters
        ----------
        n : int
            Number of successive words, default value is 1

        Returns
        -------
        dict
            Dict-like object with the 𝑛-gram string as keys and
            their counts as values

        Examples
        --------
        >>> analyzer = analyzer = PGalyzer(file_path, True)
        >>> analyzer.ngram(2)
        {'produced by': 1,
         'by greg': 1,
         'greg weeks': 1,
         'weeks dave': 1,
         ...}
        """
        par_list = self.text.split('\n')

        n_words = []

        for par in par_list:
            word_list = par.split()
            if len(word_list) >= n:
                for i in range(len(word_list)-n+1):
                    n_words.append(' '.join(word_list[i:i+n]))


        return dict(Counter(n_words).most_common())

    def word_count(self):
        """Returns the word count of the text

        Returns
        -------
        dict
            Dict-like object with the word as keys and the frequency
            as values.

        Examples
        --------
        >>> analyzer = analyzer = PGalyzer(file_path, True)
        >>> analyzer.wordcount
        {'produced': 4,
         'by': 14,
         'greg': 1,
         'weeks': 2,
         'dave': 1,
         ...}
        """
        return dict(Counter(self.text.split()).most_common())

    def concordance(self, word, neighborhood_size=10):
        """Gets the left and right neighbors of a word

        Parameters
        ----------
        word : str
            The string to be processed in getting the concordance
        neighborhood_size : int, optional
            Number of words to check from the word's left
            and right neighbors (Defaults to 10)

        Returns
        -------
        con_list : list
            List of tuples containing the left and
            right neighbors of the argument word

        Examples
        --------
        >>> analyzer = analyzer = PGalyzer(file_path, True)
        >>> analyzer.concordance('the', 2)
        [('blundell and', 'online distributed'),
         ('he was', 'first to'),
         ('of it', 'doctor fumbled')
         ...]
        """
        word_list = self.text.split('\n')
        con_list = []
        for string in word_list:
            str_list = string.split()
            indexes = [i for i, e in enumerate(str_list) if e == word]
            for i in indexes:
                if i-neighborhood_size >= 0:
                    string_before = ' '.join(str_list[i-neighborhood_size:i])
                else:
                    string_before = ' '.join(str_list[:i])

                if len(word_list) - (neighborhood_size+1) >= 0:
                    string_after = ' '.join(str_list[i+1:
                                                     i+neighborhood_size+1])
                else:
                    string_after = ' '.join(str_list[i+1:])

                con_list.append((string_before, string_after))

        return con_list

    def display_concordance(self, word, neighborhood_size=10):
        """Displays the concordance with the word in bold format

        Parameters
        ----------
        word : str
            The string to be processed in getting the concordance
        neighborhood_size : int, optional
            Number of words to check from the word's left
            and right neighbors (Defaults to 10)

        Returns
        -------
        str
            String of concordance vertically aligned on the argument
            word with the argument word in a bold format

        Examples
        --------
        >>> analyzer = analyzer = PGalyzer(file_path, True)
        >>> from IPython.core.display import HTML
        >>> HTML(analyzer.display_concordance('boys', 2))
        <pre>anything the <b>boys</b> upstairs try
        lung cancer <b>boys</b> used to
        wordand the <b>boys</b> in the</pre>
        """
        ccd = self.concordance(word, neighborhood_size)
        left_padding = max(map(lambda x: len(x[0]), ccd))
        text_list = []

        bold_word = '<b>'+word+'</b>'
        for i in ccd:
            text_list.append(f'{" " * ( left_padding - len(i[0]) )}{i[0]} '
                             f'{bold_word} {i[1]}')
        return '<pre>'+'\n'.join(text_list)+'</pre>'

    def likely_next(self, word, n=5):
        """Counter of words that are next to another word

        Parameters
        ----------
        word : str
            String to check for the words that are likely next to it.
        n : int
            Top n likely next words to retrieve

        Returns
        -------
        Counter
            Counter of top n strings that are likely next to the argument word.

        Examples
        --------
        >>> analyzer = analyzer = PGalyzer(file_path, True)
        >>> analyzer.likely_next('of')
        [('the', 30), ('his', 5), ('it', 5), ('that', 4),
        ("thurston's", 4)]
        """

        word_list = self.text.split('\n')
        d_likely_next = defaultdict(Counter)

        for wl in word_list:
            tokenized = wl.split()

            for idx, v in enumerate(tokenized[:-1]):
                d_likely_next[v].update([tokenized[idx+1]])

        return sorted(d_likely_next[word].items(),
                      key=lambda x: (-x[1], x[0]))[:n]

    def likely_previous(self, word, n=5):
        """Counter of words that are previous to another word

        Parameters
        ----------
        word : str
            String to check for the words that are likely previous to it.
        n : int
            Top n likely previous words to retrieve

        Returns
        -------
        Counter
            Counter of top n strings that are likely previous to the
            argument word.

        Examples
        --------
        >>> analyzer = analyzer = PGalyzer(file_path, True)
        >>> analyzer.likely_previous('of')
        [('out', 7), ('one', 6), ('cent', 4), ('end', 4), ('cloud', 3)]
        """
        word_list = self.text.split('\n')
        d_likely_previous = defaultdict(Counter)

        for wl in word_list:
            tokenized = wl.split()

            for idx, v in enumerate(tokenized[1:]):
                d_likely_previous[v].update([tokenized[idx]])

        return sorted(d_likely_previous[word].items(),
                      key=lambda x: (-x[1], x[0]))[:n]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


NameError: ignored

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
root = '/content/drive/My Drive/Colab Notebooks/ml1/final_project'

In [ ]:


articles = pd.read_csv(f'{root}/data/articles_v11.csv')

FileNotFoundError: ignored

In [ ]:
articles.head()

,link,category,author,published_date,updated_date,title,metadesc,happy,sad,angry,dontcare,inspired,afraid,amused,annoyed,raw
0,https://r3.rappler.com/nation/238493-preemptiv...,nation,Janella Paris,2019-08-24 23:46:00,2019-10-03 21:58:00,Preemptive measures 'spared QC from Ineng floo...,Dredging and declogging activities ordered by ...,33.0,0.0,0.0,0.0,0.0,0.0,0.0,67.0,"MANILA, Philippines – The Quezon City governme..."
1,https://r3.rappler.com/life-and-style/specials...,life-and-style,Rappler.com,2019-11-15 13:23:00,2019-11-15 13:23:00,WATCH: Emma Tiglao talks about pageant journey...,Emma is aiming for a back-to-back win in the M...,50.0,0.0,17.0,17.0,0.0,0.0,16.0,0.0,"MANILA, Philippines – The introduction video o..."
2,https://r3.rappler.com/sports/university/uaap/...,sports,JR Isaga,2019-10-07 18:44:00,2019-10-07 18:44:00,"'Marked men' Ahanmisi, Chauca not panicking am...",The Adamson Soaring Falcons have lost 3 straig...,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,"MANILA, Philippines – It seems not too long ag..."
3,https://r3.rappler.com/business/246266-naia-te...,business,Loreben Tuquero,2019-12-02 20:30:00,2019-12-02 20:30:00,NAIA terminals to close for 12 hours on Decemb...,Stranded passengers are advised to stay home a...,5.0,29.0,27.0,7.0,5.0,10.0,7.0,10.0,"MANILA, Philippines – The terminals at the Nin..."
4,https://r3.rappler.com/world/regions/asia-paci...,world,Agence France-Presse,2019-08-20 10:46:00,2019-08-20 10:46:00,Father of victim hopes for justice as Cardinal...,"Australian Cardinal George Pell, the former Va...",0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,"SYDNEY, Australia – The father of one of the v..."


In [ ]:
tx = TextAnalyzer(stop_words=stop_en, clean=True, remove_digits=True)

In [ ]:

articles['title_clean'] = articles.title.apply(lambda x : tx.fit(x).lemmatize().text)


In [ ]:
articles['metadesc_clean'] = articles.metadesc.apply(lambda x : tx.fit(x).lemmatize().text)


In [ ]:
def helper(text):
    return re.sub(r'\b([A-Z]+,[\w\s]+–)\s?|– Rappler.com', '', text)

articles['raw_clean'] = articles.raw.apply(lambda x : tx.fit(helper(x)).lemmatize().text)

In [ ]:
articles.head()

,link,category,author,published_date,updated_date,title,metadesc,happy,sad,angry,dontcare,inspired,afraid,amused,annoyed,raw,title_clean,metadesc_clean,raw_clean
0,https://r3.rappler.com/nation/238493-preemptiv...,nation,Janella Paris,2019-08-24 23:46:00,2019-10-03 21:58:00,Preemptive measures 'spared QC from Ineng floo...,Dredging and declogging activities ordered by ...,33.0,0.0,0.0,0.0,0.0,0.0,0.0,67.0,"MANILA, Philippines – The Quezon City governme...",preemptive measure spar qc ineng flood city gov,dredge declogging activity order quezon city m...,quezon city government report late saturday ni...
1,https://r3.rappler.com/life-and-style/specials...,life-and-style,Rappler.com,2019-11-15 13:23:00,2019-11-15 13:23:00,WATCH: Emma Tiglao talks about pageant journey...,Emma is aiming for a back-to-back win in the M...,50.0,0.0,17.0,17.0,0.0,0.0,16.0,0.0,"MANILA, Philippines – The introduction video o...",watch emma tiglao talk pageant journey charity...,emma aim back back win miss intercontinental p...,introduction video emma tiglao philippine repr...
2,https://r3.rappler.com/sports/university/uaap/...,sports,JR Isaga,2019-10-07 18:44:00,2019-10-07 18:44:00,"'Marked men' Ahanmisi, Chauca not panicking am...",The Adamson Soaring Falcons have lost 3 straig...,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,"MANILA, Philippines – It seems not too long ag...",marked men ahanmisi chauca panic amid adamson ...,adamson soar falcon lose straight game win fir...,seem long ago adamson soar falcon cusp reach u...
3,https://r3.rappler.com/business/246266-naia-te...,business,Loreben Tuquero,2019-12-02 20:30:00,2019-12-02 20:30:00,NAIA terminals to close for 12 hours on Decemb...,Stranded passengers are advised to stay home a...,5.0,29.0,27.0,7.0,5.0,10.0,7.0,10.0,"MANILA, Philippines – The terminals at the Nin...",naia terminal close hour december due typhoon ...,strand passenger advise stay home avoid go air...,terminal ninoy aquino international airport na...
4,https://r3.rappler.com/world/regions/asia-paci...,world,Agence France-Presse,2019-08-20 10:46:00,2019-08-20 10:46:00,Father of victim hopes for justice as Cardinal...,"Australian Cardinal George Pell, the former Va...",0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,"SYDNEY, Australia – The father of one of the v...",father victim hope justice cardinal pell wait ...,australian cardinal george pell former vatican...,father one victim jail australian cardinal geo...


In [ ]:
articles.isnull().sum()

link              0
category          0
author            0
published_date    0
updated_date      0
title             0
metadesc          0
happy             0
sad               0
angry             0
dontcare          0
inspired          0
afraid            0
amused            0
annoyed           0
raw               0
title_clean       0
metadesc_clean    4
raw_clean         0
dtype: int64

In [ ]:
text = articles.raw_clean[0]
text.split('\n')

['quezon city government report late saturday night august preemptive measure help spare city massive flood experienced part metro manila follow heavy rain cause southwest monsoon',
 'qc government say press statement base assessment fact report massive flood throughout saturday relayed quezon city emergency operation center',
 'add qc urban search rescue team place standby receive distress call relation flood',
 'accord quezon city disaster risk reduction management office drrmo head karl michael marasigan preemptive measure order qc mayor joy belmonte declogging dredge activity city early july clear city drainage system help prevent flood',
 'round clock declogging canal order mayor joy belmonte take advantage two week monsoon break dredge key waterway traverse city since first week belmonte administration facilitate speedy flow rainwater drainage system marasigan say',
 '',
 'qc government note pagasa science garden synoptic station record millimeter rain saturday classify heavy hou

In [ ]:
len(re.findall(r'\d+', ' '.join(articles.raw_clean.values)))

0

In [ ]:
articles.to_csv(f'{root}/data/articles_v7.csv', index=False)

In [ ]:
articles1 = pd.read_csv(f'{root}/data/articles_v12.csv')

In [ ]:
articles1[(articles1.happy >= 20) & (articles1.sad >= 15) & (articles1.afraid >= 10) & (articles1.angry >= 10) & (articles1.inspired >= 10)].iloc[0,:].title

'Tisoy weakens into tropical depression as it leaves PAR'

In [ ]:
articles1[(articles1.happy >= 20) & (articles1.sad >= 15) & (articles1.afraid >= 10) & (articles1.angry >= 10) & (articles1.inspired >= 10)].iloc[0,:].metadesc

'Though Tisoy (Kammuri) is no longer inside the Philippine Area of Responsibility, there may still be heavy rain due to the surge of the northeast monsoon and the tail-end of a cold front'

In [ ]:
articles1[(articles1.happy >= 20) & (articles1.sad >= 15) & (articles1.afraid >= 10) & (articles1.angry >= 10) & (articles1.inspired >= 10)].iloc[0,:]

link                https://r3.rappler.com/nation/special-coverage...
category                                                       nation
author                                                     Acor Arceo
published_date                                    2019-12-05 12:30:00
updated_date                                      2019-12-05 12:30:00
title               Tisoy weakens into tropical depression as it l...
metadesc            Though Tisoy (Kammuri) is no longer inside the...
happy                                                              22
sad                                                                33
angry                                                              11
dontcare                                                            0
inspired                                                           11
afraid                                                             11
amused                                                              1
annoyed             

In [ ]:
articles1[articles1.raw_clean.isnull()].raw.iloc[0]

['0750. ']

In [ ]:
replace = {'– Rappler.com', ''}



["Autism awareness and mental health are some of the causes closest to this beauty queen's heart"]

In [ ]:
text = articles1.raw.iloc[0]
text

'MANILA, Philippines – The Quezon City government reported late Saturday night, August 24, that the its preemptive measures helped spare the city from the massive flooding experienced in other parts of Metro Manila, following heavy rains caused by the southwest monsoon.\nThe QC government said in a press statement that it based this assessment on the fact that "there were no reports of massive flooding throughout Saturday," as relayed by the  Quezon City Emergency Operations Center.\nIt added that the QC Urban Search and Rescue Team which had been placed on standby did not receive any distress call in relation to flooding.\nAccording to Quezon City Disaster Risk Reduction and Management Office (DRRMO) head Karl Michael Marasigan, preemptive measures ordered by QC Mayor Joy Belmonte – declogging and dredging activities in the city as early as July – had cleared the city\'s drainage system and helped prevent flooding.\n“The round-the-clock declogging of canals ordered by Mayor Joy Belmon

In [ ]:
re.sub(r'– Rappler.com', '', text)

'MANILA, Philippines – The Quezon City government reported late Saturday night, August 24, that the its preemptive measures helped spare the city from the massive flooding experienced in other parts of Metro Manila, following heavy rains caused by the southwest monsoon.\nThe QC government said in a press statement that it based this assessment on the fact that "there were no reports of massive flooding throughout Saturday," as relayed by the  Quezon City Emergency Operations Center.\nIt added that the QC Urban Search and Rescue Team which had been placed on standby did not receive any distress call in relation to flooding.\nAccording to Quezon City Disaster Risk Reduction and Management Office (DRRMO) head Karl Michael Marasigan, preemptive measures ordered by QC Mayor Joy Belmonte – declogging and dredging activities in the city as early as July – had cleared the city\'s drainage system and helped prevent flooding.\n“The round-the-clock declogging of canals ordered by Mayor Joy Belmon

'The Quezon City government reported late Saturday night, August 24, that the its preemptive measures helped spare the city from the massive flooding experienced in other parts of Metro Manila, following heavy rains caused by the southwest monsoon.\nThe QC government said in a press statement that it based this assessment on the fact that "there were no reports of massive flooding throughout Saturday," as relayed by the  Quezon City Emergency Operations Center.\nIt added that the QC Urban Search and Rescue Team which had been placed on standby did not receive any distress call in relation to flooding.\nAccording to Quezon City Disaster Risk Reduction and Management Office (DRRMO) head Karl Michael Marasigan, preemptive measures ordered by QC Mayor Joy Belmonte – declogging and dredging activities in the city as early as July – had cleared the city\'s drainage system and helped prevent flooding.\n“The round-the-clock declogging of canals ordered by Mayor Joy Belmonte to take advantage o